# Gaussian Bayesian Classifier

### Mastery Checkpoint 1  
CSC 466, Winter 2022  
Samay Nathani

In [1]:
from pathlib import Path
home = str(Path.home())

In [2]:
%load_ext autoreload
%autoreload 2

import Lab2_helper

Import our dataset
I chose the [Uber Fares Dataset from Kaggle](https://www.kaggle.com/yasserh/uber-fares-dataset/version/1)

In [3]:
import pandas as pd
import numpy as np
import math
raw_uber_df = pd.read_csv(
    f"uber.csv"
)

Do some preprocessing like Lab 2

In [4]:
features = ['pickup_longitude','pickup_latitude','dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'fare_amount']
raw_uber_df = raw_uber_df.loc[:,features]
display(raw_uber_df.head())

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,fare_amount
0,-73.999817,40.738354,-73.999512,40.723217,1,7.5
1,-73.994355,40.728225,-73.994710,40.750325,1,7.7
2,-74.005043,40.740770,-73.962565,40.772647,1,12.9
3,-73.976124,40.790844,-73.965316,40.803349,3,5.3
4,-73.925023,40.744085,-73.973082,40.761247,5,16.0


Creating a function to find the distance between the drop-off and pick-up locations, credit goes to [Geeks4Geeks]()

In [5]:
def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = math.radians(lon1)
    lon2 = math.radians(lon2)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
 
    c = 2 * math.asin(math.sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)

Do some preprocessing to get the distance, passenger count, and fare of an uber

In [6]:
raw_uber_df.dropna(inplace=True)
raw_uber_df['distance'] = raw_uber_df.apply(lambda x: distance(x.loc['pickup_latitude'], x.loc['dropoff_latitude'], x.loc['pickup_longitude'], x.loc['dropoff_longitude']), axis=1)
final_features = ['distance', 'passenger_count', 'fare_amount']
uber_df = raw_uber_df.loc[:, final_features]
uber_df.head()

,distance,passenger_count,fare_amount
0,1.683323,1,7.5
1,2.457590,1,7.7
2,5.036377,1,12.9
3,1.661683,3,5.3
4,4.475450,5,16.0


In [7]:
uber_df.loc[:, 'distance'] = round(uber_df['distance'])
uber_df.loc[:, 'fare_amount'] = round(uber_df['fare_amount'])
uber_df.head()

,distance,passenger_count,fare_amount
0,2.0,1,8.0
1,2.0,1,8.0
2,5.0,1,13.0
3,2.0,3,5.0
4,4.0,5,16.0


In [8]:
"""
titanic_df.loc[:,'pickup_longitude']=titanic_df['Pclass'].fillna(titanic_df['Pclass'].mode()).astype(int)
titanic_df.loc[:,'pickup_latitude']=titanic_df['Age'].fillna(titanic_df['Age'].median())
titanic_df.loc[:,'Age']=(titanic_df['Age']/10).astype(str).str[0].astype(int)*10
titranic_df = titanic_df.dropna()
X = titanic_df.drop("Survived",axis=1)
y = titanic_df["Survived"]
#probs = Lab2_helper.class_conditional(X,y)
#priors = Lab2_helper.compute_priors(y)
x = titanic_df.drop("Survived",axis=1).loc[2]
#post_probs = Lab2_helper.posteriors(probs,priors,x)
"""

'\ntitanic_df.loc[:,\'pickup_longitude\']=titanic_df[\'Pclass\'].fillna(titanic_df[\'Pclass\'].mode()).astype(int)\ntitanic_df.loc[:,\'pickup_latitude\']=titanic_df[\'Age\'].fillna(titanic_df[\'Age\'].median())\ntitanic_df.loc[:,\'Age\']=(titanic_df[\'Age\']/10).astype(str).str[0].astype(int)*10\ntitranic_df = titanic_df.dropna()\nX = titanic_df.drop("Survived",axis=1)\ny = titanic_df["Survived"]\n#probs = Lab2_helper.class_conditional(X,y)\n#priors = Lab2_helper.compute_priors(y)\nx = titanic_df.drop("Survived",axis=1).loc[2]\n#post_probs = Lab2_helper.posteriors(probs,priors,x)\n'

Defining the Gaussian Bayesian Classifier

In [86]:
class GaussianBayes:
    
    def compute_priors(self, y):
        value_counts = y.value_counts().sort_index()
        total_values = len(y)
        indexes = [y.name+"="+str(v) for v in value_counts.index.tolist()]
        priors = dict(zip(indexes, value_counts / total_values))
        return priors
    
    def compute_normal_distribution(self, mean, stddev, x):
        exp = (((x-mean) / stddev)**2) * -0.5
        denom = ((stddev) * math.sqrt(2 * math.pi))
        return (math.e ** exp) / denom
    
    
    # Since we are assuming a normal distribution, this is where we use the mean, std, and variance to calculate the class conditional probability. 
    def specific_class_conditional(self,x,xv,y,yv):
        # todo: replace binning with normal distribution calculations
        likelihoods = {}
        priors = {}
        for ux in x.unique():
            y_vals = y.loc[x==ux]
            for uy in y_vals.unique():
                if not math.isnan(y_vals.std()):
                    prob = self.compute_normal_distribution(y_vals.mean(), np.nanstd(y_vals), uy)
                    likelihoods[str(ux) + "|" + str(uy)] = prob
        print(likelihoods)
        return likelihoods[str(xv) + "|" + str(yv)]


    def class_conditional(self,X,y):
        probs = {}

        for eachy in y.unique():
            for col in X.columns:
                for eachx in X[col].unique():
                    probs[col + "=" + str(eachx) + "|" + y.name + "=" + str(eachy)] = self.specific_class_conditional(X[col], eachx, y, eachy)
        return probs

    def posteriors(self,probs,priors,x):
        post_probs = {}
        denom = 0
        for k in priors.keys():
            numerator = 1
            postkey = ""
            
            for idx in x.index:
                postkey += idx + "=" + str(x[idx]) + ","
                probkey = idx + "=" + str(x[idx]) + "|" + k
                
                if probkey not in probs:
                    numerator *= 0
                else:
                    numerator *= probs[probkey]
            
            numerator*=priors[k]
            post_probs[k + "|" + postkey[:-1]] = numerator
            denom+=numerator
            
        for k, v in post_probs.items():
            if denom != 0:
                post_probs[k] = v / denom
            else:
                post_probs[k] = 1 / len(list(priors.keys()))
        
        return post_probs


In [87]:
gb = GaussianBayes()

In [88]:
prob = gb.specific_class_conditional(uber_df['distance'], 3, uber_df['fare_amount'], 8)
prob

{'2.0|8.0': 0.15188559490270628, '2.0|5.0': 0.07899440029621355, '2.0|10.0': 0.11377453529802187, '2.0|6.0': 0.1135498958946258, '2.0|12.0': 0.04772788523533236, '2.0|7.0': 0.14119744358365022, '2.0|9.0': 0.14133704229177713, '2.0|11.0': 0.07922893193216729, '2.0|4.0': 0.04753960093833383, '2.0|13.0': 0.02487197322580232, '2.0|14.0': 0.011212383683166108, '2.0|18.0': 0.0001086789369065373, '2.0|17.0': 0.0004304860025294585, '2.0|3.0': 0.02474938528766565, '2.0|15.0': 0.004372555386396058, '2.0|58.0': 3.1938900563937837e-80, '2.0|16.0': 0.0014751030035856615, '2.0|71.0': 1.8624488950201766e-126, '2.0|26.0': 9.714576986587877e-12, '2.0|19.0': 2.3734571077502602e-05, '2.0|23.0': 1.267132431377555e-08, '2.0|22.0': 1.0360658587334854e-07, '2.0|2.0': 0.011146100660109542, '2.0|20.0': 4.484014964701625e-06, '2.0|30.0': 8.958905928627611e-17, '2.0|33.0': 3.2726511326765595e-21, '2.0|35.0': 1.7454650830923386e-24, '2.0|32.0': 1.1401670178535102e-19, '2.0|50.0': 4.659385489171236e-57, '2.0|28.0'

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


KeyError: '3|8'